# Pixel Drill
This section demonstrates the usage of a Jupyter Notebook to extract time series values of a given pixel from the specified datasets over a specified time period to a CSV file. It works for different dataset types available in DEA currently, including NBAR (Nadir BRDF (Bidirectional Reflectance Distribution Function) Adjusted Reflectance), NBART (NBAR plus Terrain illumination correction), FC (Fractional Cover) and NDVI (Normalized Difference Vegetation Index).

Firstly load the required modules:

In [1]:
%matplotlib inline
import datacube
from datetime import datetime
from datacube.storage import masking
import numpy as np
import pandas
import xarray as xr

Then set the pixel location, time period, product type and satellite platform you are interested. These input parameters are highlighted in yellow to show that they are controlled by the user.


In [2]:
lon = 149
lat = -37
acq_min = '2005-1'
acq_max = '2015-12'
product_type = 'ndvi'
platform_list = ['ls5', 'ls7', 'ls8']


In the above product type, other options are NBAR, NBART and FC. Based on the input parameters, the output csv file name is defined as


In [3]:
output_name = '{}_{}_{}_{}_{}_{}.csv'.format('_'.join(platform_list), product_type, 
                                             str(lon), str(lat), acq_min, acq_max)


which contains the platform, product type, spatial location and time range specified.
Before performing the pixel drill function in this example, there is a support function for retrieving pixel quality data, which needs to be run in the notebook. 
This function is temporary here and will be in Data Cube API in the future, which means it will disappear in this notebook sometime in the future.


In [4]:
valid_bit = 8
def pq_fuser(dest, src): 
    valid_val = (1 << valid_bit) 

    no_data_dest_mask = ~(dest & valid_val).astype(bool) 
    np.copyto(dest, src, where=no_data_dest_mask) 
    both_data_mask = (valid_val & dest & src).astype(bool) 
    np.copyto(dest, src & dest, where=both_data_mask)


The following function is used to retrieve data for a pixel from a specified product with a specific location and time range. The output of this function is the cloud free and valid data (i.e. without nodata) value.



In [5]:
def get_pix_cloud_free_valid(product_type, platform, lon, lat, acq_min, acq_max):
    # get the product name and pq product name
    product_name = '{}_{}_albers'.format(platform, product_type)
    pq_name = '{}_pq_albers'.format(platform)
    #print product_name
    
    # use a small buffer to load data, as single pixel data is not allowed to load at this moment
    if product_type  in ['nbar', 'nbart']:
        # select measurements for NBAR/NBART, so that LS8 data can merge with others
        measurements_list = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
        buff = dc.load(product=product_name, 
                   measurements=measurements_list,
                   x=(lon, lon+0.00025), y=(lat, lat+0.00025), 
                   time=(acq_min, acq_max), 
                   group_by='solar_day')
    else:        
        buff = dc.load(product=product_name, 
                   x=(lon, lon+0.00025), y=(lat, lat+0.00025), 
                   time=(acq_min, acq_max), 
                   group_by='solar_day')
    if buff:
        # select the pixel of interest
        pix = buff.isel(x=0, y=0)
        # get cloud free data for the pixel of interest
        pq = dc.load(product=pq_name, x=(lon, lon+0.00025), y=(lat, lat+0.00025), time=(acq_min, acq_max), 
                 group_by='solar_day', fuse_func=pq_fuser)
        pq = pq.isel(x=0, y=0)
        cloud_free = masking.make_mask(pq, cloud_acca='no_cloud', cloud_fmask='no_cloud', contiguous=True).pixelquality
        pix_cld_free = pix.where(cloud_free).dropna(dim='time')
        # remove nodata for the pixel of interest
        pix_cld_free_valid = masking.mask_valid_data(pix_cld_free)
        
        pix_cld_free_valid['platform'] = (('time'), [platform]*pix_cld_free_valid.time.size)
    
        return pix_cld_free_valid
    else:
        return
                     

The following is the break-down explanation of the above function:

1. We need to define which product in the DEA will be used for pixel drill according to the user inputs (see the orange rectangle part labelled with 1). 

2. If the product type is NBAR or NBART, because Landsat 8 has different bands compared to Landsat 7 or Landsat 5, we need to explicitly define the common bands (blue, red, green, Near Infrared (nir), Short-wave Infrared 1 (swir1) and Short-wave Infrared 2 (swir2)) between the Landsat platforms. In doing this, the pixel information for each of the common bands for all Landsat platforms can be merged to form the final product (see the orange rectangle part labelled with 2).  

3. At this moment, it is not possible to load data for a single pixel, we need to use a small buffer (e.g. adding another pixel in this example) to load data, and then select the data of the pixel of interest from the buffer (see the three orange rectangle parts labelled with 3).

4. The last part of this function is to apply pixel quality mask to the retrieved raw data, so that only the data with cloud free and valid values is returned (see the orange rectangle part labelled with 4).

Now we initialise a Datacube object that connects to DEA, giving an application name as px_drill.


In [6]:
dc = datacube.Datacube(app='px_drill')


We go through each platform (LS5, LS7 or LS8) of interest to retrieve the cloud free and valid pixel data. The output is put into a list for merging later on.

In [7]:
pixel_data_list = []
for platform in platform_list:
    pixel_data = get_pix_cloud_free_valid(product_type, platform, lon, lat, acq_min, acq_max)        
    if pixel_data:
        pixel_data_list.append(pixel_data)


Then we merge the products into a final one incorporating all platforms of interest. The final product is sorted against time. For the purpose of demonstration, the code also plots the first band information over the time period for the final product.


In [ ]:
pixel_merge = xr.concat(pixel_data_list, dim='time')
time_sorted = pixel_merge.time.argsort()
pixel_merge_sorted = pixel_merge.isel(time=time_sorted)
pixel_merge_sorted.data_vars.itervalues().next().plot()


The final product is organised into a pandas dataframe, which can be displayed on the screen and exported to a csv file as well. This is the code:


In [ ]:
df = pixel_merge_sorted.to_dataframe()
df = df.drop(['x', 'y'], axis=1)
cols = list(df)
cols.insert(0, cols.pop(cols.index('platform')))
df = df[cols]
df.to_csv(output_name)


The output csv files name contains all relevant input parameters such as the satellite platform, product type, pixel location and the range of acquisition date. In this example, the csv file name is ls5_ls7_ls8_ndvi_149_-37_2005-1_2015_12.csv.

If you would like to perform a pixel drill for other product types such as FC, NBAR or NBART, only thing to do is replacing ndvi in product_type = 'ndvi' with fc, nbar or nbart, all other corresponding information including band information and output file will automatically apply to the specific product type.  
To view these csv files, you can use FileZilla software to transfer the files from NCI to your local working directory. To connect to VDI within FileZilla, in the host box, input sftp://hostname.nci.org.au, and 22 in the Port box while the hostname can be found in the very top of a terminal window in VDI or by typing a command (hostname) in a terminal window to find out.